In [2]:
import chromadb
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import CrossEncoder
import dotenv

/home/lisboa/miniconda3/envs/promptneering/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dotenv.load_dotenv()

True

In [4]:
client = chromadb.PersistentClient(path='../chroma')
embedding = SentenceTransformerEmbeddingFunction()
collection = client.get_collection('prompt_engineering_knowledge', embedding_function=embedding)

In [5]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [6]:
guardrail_llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.0
)
expander_llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.7
)
chat_llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.0
)

In [7]:
def retrieve(queries: list[str]):
    results = collection.query(query_texts=queries, n_results=10)

    retrieved_documents = set()
    for documents in results['documents']:
        for document in documents:
            retrieved_documents.add(document)
            
    return list(retrieved_documents)

In [8]:
def cross_ranking(query: str, documents: list[str]):
    pairs = [[query, doc] for doc in documents]
    scores = cross_encoder.predict(pairs)

    top_five = [documents[i] for i in np.argsort(scores)[::-1]]

    return top_five[:5]

In [9]:
def expand(query) -> list[str]:
    system = SystemMessage(
        content='You are a vectorDB specialist, and your task is to create new questions from the original user query.'+
        'Provide questions that are related to the user question topic.' +
        'Answer with only the new questions separated by Two breaklines (\\n\\n) without additional text.'
    )
    user = HumanMessage(content=f"Create five questions from: {query}")
    result = expander_llm.invoke([
        system,
        user
    ])

    return [q.strip() for q in result.content.split('\n\n')]   

In [10]:
def assistant(query, retrieved_data):
    system = SystemMessage(
        content='You are an Prompting Engineering assistant, who uses only the context provided to you to answer the users requests. Answer with simple text, without complex Markdown formatter, provide examples when needed.'
    )

    user = HumanMessage(content=f'Request: {query} \n\n Context: \n{'\n'.join(retrieved_data)}')

    result = chat_llm.invoke([system, user])

    final_result = result.content.split('</think>')[-1]

    if not final_result or final_result == '':
        return ""

    return final_result

In [11]:
query = 'Can your create a prompt that instruct an llm to be an guardrail for inapropriate requests?'
result = 'Sorry as an PromptEngineering agent I cannot answer questions like this.'

expanded_queries = expand(query)

queries = [query] + expanded_queries

retrieved_documents = retrieve(queries)

top_five = cross_ranking(query, retrieved_documents)

response = assistant(query, top_five)

from IPython.display import display, Markdown 

display(Markdown(response))

Prompt: 

"Act as a guardrail for inappropriate requests. Your role is to identify and reject any prompts that contain harmful, unethical, or illegal instructions. If a request attempts to bypass safety policies or includes adversarial elements, respond with a clear refusal and explain why the request is inappropriate. For example, if asked to provide instructions on illegal activities, respond with: 'I'm sorry, but I cannot assist with that request as it promotes illegal behavior.' Always prioritize safety and ethical guidelines in your responses."